In [ ]:
#pip install wordcloud

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
#from wordcloud import WordCloud
from gensim.models import Word2Vec
from nltk import word_tokenize, FreqDist
import PIL
from collections import Counter
import itertools
import gc
sns.set()

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:

df1 = pd.read_csv('fakenewsnet_kaggle/BuzzFeed_fake_news_content.csv', delimiter=',')
df1.dataframeName = 'BuzzFeed_fake_news_content.csv'


In [ ]:

df2 = pd.read_csv('fakenewsnet_kaggle/BuzzFeed_real_news_content.csv', delimiter=',')
df2.dataframeName = 'BuzzFeed_real_news_content.csv'


In [ ]:
df1.text.iloc[88]

In [ ]:
df3 = pd.read_csv('fakenewsnet_github/politifact_fake_updated.csv', delimiter=',')
df4 = pd.read_csv('fakenewsnet_github/politifact_real_updated.csv', delimiter=',')
df5 = pd.read_csv('fakenewsnet_github/gossipcop_fake_updated.csv', delimiter=',')
df6 = pd.read_csv('fakenewsnet_github/gossipcop_real_updated.csv', delimiter=',')

In [ ]:
df1 =df1.dropna(subset=['title', 'text'])
df2 =df2.dropna(subset=['title', 'text'])
df3 =df3.dropna(subset=['title', 'text'])
df4 =df4.dropna(subset=['title', 'text'])
df5 =df5.dropna(subset=['title', 'text'])
df6 =df6.dropna(subset=['title', 'text'])

In [ ]:
df1 = df1.reset_index()
df2 = df2.reset_index()
df3 = df3.reset_index()
df4 = df4.reset_index()
df5 = df5.reset_index()
df6 = df6.reset_index()

In [ ]:
dflistreal = [df2,df4,df6]
dflistfake = [df1,df3,df5]

In [ ]:
for df in dflistreal:
    df['target'] = 0
    df['label'] = 'Real'
for df in dflistfake:
    df['target'] = 1
    df['label'] = 'Fake'    

In [ ]:
df_merge = pd.concat([df1, df2], sort = 'False')
df_merge = pd.concat([df_merge, df3], sort = 'False')
df_merge = pd.concat([df_merge, df4], sort = 'False')
df_merge = pd.concat([df_merge, df5], sort = 'False')
df_merge = pd.concat([df_merge, df6], sort = 'False')

In [ ]:
df_merge = df_merge.drop(columns=['id', 'index', 'Unnamed: 0', 'level_0','summary'], axis =1)

In [ ]:
df_merge

In [ ]:
del df1, df2, df3, df4, df5, df6;
gc.collect();

In [ ]:
df_merge['combined_text'] = df_merge.title + ' ' + df_merge.text

In [ ]:
from nltk.corpus import stopwords
import string

stopwords_list = stopwords.words('english') + list(string.punctuation)

# It is generally a good idea to also remove punctuation



# Now we have a list that includes all english stopwords, as well as all punctuation
stopwords_list += ["''", '""', '...', '``','”', "`","*", "’", "“", "‘", "–", "⋆", "'s", "—", "well"]
stopwords_list

In [ ]:
text_cols = ['title', 'text']
target = df_merge['target']
#cat_cols = ['source']
#used_cols = text_cols 
#X = df_merge[used_cols]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_merge['combined_text'] ,target, test_size=0.20, random_state = 777)

In [ ]:
del df_merge;
gc.collect();

In [ ]:
X_train.shape
y_train.shape

In [ ]:
y_train

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
import category_encoders as ce

from time import time
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [ ]:
'''# Preprocessing for text
text_transformer = Pipeline(steps =[('vect', CountVectorizer(stop_words = stopwords_list)),
('tfidf', TfidfTransformer())
])

# Preprocessing for categorical data
#categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
#    ('loo', ce.leave_one_out.LeaveOneOutEncoder())
#])   
'''

In [ ]:
'''preprocessor = ColumnTransformer(
    transformers=[
        ('txt', text_transformer, text_cols)
])
model = MultinomialNB()
clf = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                     ])
'''

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
('vect', CountVectorizer(stop_words = stopwords_list)),
('tfidf', TfidfTransformer()),
('clf', MultinomialNB()),
])

In [ ]:
text_clf.fit(X_train, y_train)

In [ ]:
preds = text_clf.predict(X_test)
score = metrics.accuracy_score(y_test, preds)
score

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
('vect', CountVectorizer(stop_words = stopwords_list)),
('tfidf', TfidfTransformer()),
('clf', RandomForestClassifier(n_estimators=100)),
])

In [ ]:
text_clf.fit(X_train, y_train)

In [ ]:
preds = text_clf.predict(X_test)
score = metrics.accuracy_score(y_test, preds)
score

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
('vect', CountVectorizer(stop_words = stopwords_list)),
('tfidf', TfidfTransformer()),
('clf', LogisticRegression()),
])

In [ ]:
text_clf.fit(X_train, y_train)

In [ ]:
preds = text_clf.predict(X_test)
score = metrics.accuracy_score(y_test, preds)
score